# LCEL & RUNNABLES


## SETUP


In [1]:
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

In [5]:
# for langchain debugging
from langchain_core.globals import set_debug

set_debug(False)

## PASS DATA THROUGH


In [13]:
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    passed=RunnablePassthrough(),  # 1
    extra=RunnablePassthrough.assign(
        mult=lambda x: x["num"] * 3,  # 3
    ),
    modified=lambda x: x["num"] + 1,  # 2
)

runnable.invoke({"num": 1})

{'passed': {'num': 1}, 'extra': {'num': 1, 'mult': 3}, 'modified': 2}

In [17]:
# custom functions
from operator import itemgetter

# from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableLambda

In [18]:
def length_function(text):
    return len(text)


def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)


def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])

In [19]:
prompt = ChatPromptTemplate.from_template("what is {a} + {b}")

llm = ChatOpenAI(api_key=OPENAI_API_KEY, model="gpt-3.5-turbo", temperature=0)

In [39]:
chain = (
    {
        "a": itemgetter("foo") | RunnableLambda(length_function),  # 9
        "b": {
            "text1": itemgetter("foo"),
            "text2": itemgetter("bar"),
        }
        | RunnableLambda(multiple_length_function),
    }
    | prompt
    | llm
)

In [40]:
chain.invoke({"foo": "bartender", "bar": "garden"})

AIMessage(content='9 + 54 = 63', response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 14, 'total_tokens': 21}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-55ba4695-335f-4e15-9daa-e8e72554340a-0')

## ACCEPTING RUNNABLE CONFIGS

In [41]:
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableConfig
import json

In [43]:
def parse_or_fix(text: str, config: RunnableConfig):
    fixing_chain = (
        ChatPromptTemplate.from_template(
            "Fix the following text:\n\n```text\n{input}\n```\nError: {error}"
            " Don't narrate, just respond with the fixed data."
        )
        | llm
        | StrOutputParser()
    )
    for _ in range(3):
        try:
            return json.loads(text)
        except Exception as e:
            text = fixing_chain.invoke(
                {
                    "input": text,
                    "error": e,
                },
                config,
            )
    return "Failed to parse"

In [49]:
from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:
    output = RunnableLambda(parse_or_fix).invoke(
        "{foo: bar}", {"tags": ["my-tag"], "callbacks": [cb]}
    )
    print(output)
    print(cb)

{'foo': 'bar'}
Tokens Used: 62
	Prompt Tokens: 56
	Completion Tokens: 6
Successful Requests: 1
Total Cost (USD): $9.6e-05


## DYNAMIC ROUTING BASED ON INPUT

### Routing w/ Custom function

In [82]:
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser

In [83]:
# prompts
promt_router = PromptTemplate.from_template(
    """Given the user question below, classify it as either being about `Bodybuilding`, `Jungian Psychology`, or `Other`.

Do not respond with more than one word.

<question>
{question}
</question>

Classification:"""
)

prompt_body_building = PromptTemplate.from_template(
    """You are an expert in bodybuilding Always answer questions starting with "As Dr. Mike Israetel from Rennaissance Periodization tol me".

    Respond to the following question:
    Question: {question}
    Answer:"""
)

prompt_jungian = PromptTemplate.from_template(
    """You are an expert in Jungian Psychology and Theory. \
    Always answer questions starting with "As Dr. C.G. Jung would say". \
    Respond to the following question:

    Question: {question}
    Answer:"""
)

prompt_general = PromptTemplate.from_template(
    """Respond to the following question:

Question: {question}
Answer:"""
)

In [89]:
# chains
chain_router = promt_router | llm | StrOutputParser()
chain_body_building = prompt_body_building | llm
chain_jungian = prompt_jungian | llm
chain_general = prompt_general | llm

In [90]:
def routing_function(info):
    if "bodybuilding" in info["topic"].lower():
        return chain_body_building
    elif "jungian psychology" in info["topic"].lower():
        return chain_jungian
    else:
        return chain_general

In [95]:
from langchain.schema.runnable import RunnableLambda

chain_full = {
    "topic": chain_router,
    "question": lambda x: x["question"],
} | RunnableLambda(routing_function)

In [96]:
chain_full.invoke({"question": "what does alchemy have to do with dreams?"})

AIMessage(content="As Dr. C.G. Jung would say, alchemy and dreams are interconnected through the concept of transformation. Alchemy is a symbolic process of inner transformation, where base materials are transformed into gold, representing the individuation process. Dreams, on the other hand, are symbolic expressions of the unconscious mind, often containing images and symbols that reflect the individual's inner conflicts and desires. By exploring and interpreting dreams, one can uncover hidden aspects of the self and work towards psychological integration, similar to the alchemical process of transmutation.", response_metadata={'token_usage': {'completion_tokens': 106, 'prompt_tokens': 58, 'total_tokens': 164}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-2d54742b-f713-4c60-a998-35c412f074b2-0')

## BINDING RUNTIME ARGUMENTS


In [97]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

In [106]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Write out 4 flashcard questions with their options based on the topic given below.. Use the format\n Question:...\nOPTIONS:...\nSOLUTION:...\n\n",
        ),
        ("human", "{topic}"),
    ]
)

In [108]:
chain = {"topic": RunnablePassthrough()} | prompt | llm | StrOutputParser()

print(chain.invoke("langchain runnables"))

Question: What is the purpose of using runnables in Langchain?
OPTIONS: A) To execute code in a separate thread B) To handle exceptions C) To define custom data types D) To interact with databases
SOLUTION: A) To execute code in a separate thread

Question: How are runnables implemented in Langchain?
OPTIONS: A) Using interfaces B) Using classes C) Using annotations D) Using enums
SOLUTION: A) Using interfaces

Question: Which method needs to be implemented when creating a runnable in Langchain?
OPTIONS: A) run() B) start() C) execute() D) process()
SOLUTION: A) run()

Question: What is the benefit of using runnables in Langchain?
OPTIONS: A) Improved performance B) Better memory management C) Easier debugging D) Enhanced security
SOLUTION: A) Improved performance


In [111]:
chain_with_bind = (
    {"topic": RunnablePassthrough()}
    | prompt
    | llm.bind(stop="SOLUTION")
    | StrOutputParser()
)

print(chain_with_bind.invoke("langchain runnables"))

Question: What is the purpose of using runnables in Langchain?
OPTIONS: A) To execute code concurrently B) To store data C) To create graphical user interfaces D) To handle exceptions



## OPENAI FUNCTIONS

In [113]:
function = {
    "name": "return_questions",
    "descriptions": "extracts questions from raw text",
    "parameters": {
        "type": "object",
        "properties": {
            "raw_text": {
                "type": "string",
                "description": "The raw text of flashcard questions",
            },
            "question": {
                "type": "array",
                "description": "array of questions",
                "items": {
                    "type": "string",
                    "description": "question string",
                },
            },
        },
        "required": ["equation", "solution"],
    },
}

In [114]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Write out flashcard questions to the following topic then return the list of questions.",
        ),
        ("human", "{topic}"),
    ]
)

In [115]:
from langchain.chat_models import ChatOpenAI

In [117]:
llm_gpt4 = ChatOpenAI(model="gpt-4o", temperature=0).bind(
    function_call={"name": "return_questions"}, functions=[function]
)
runnable = {"topic": RunnablePassthrough()} | prompt | llm_gpt4
runnable.invoke("world war 2")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"raw_text":"1. What were the main causes of World War 2?\\n2. Which countries were part of the Axis Powers?\\n3. Which countries were part of the Allied Powers?\\n4. What event marked the beginning of World War 2?\\n5. Who was the leader of Nazi Germany during World War 2?\\n6. What was the significance of the Battle of Stalingrad?\\n7. What was the purpose of the D-Day invasion?\\n8. What were the major outcomes of the Yalta Conference?\\n9. How did World War 2 end in Europe?\\n10. What were the consequences of the atomic bombings of Hiroshima and Nagasaki?\\n11. What was the Holocaust?\\n12. How did World War 2 impact the global balance of power?\\n13. What was the Marshall Plan?\\n14. What were the Nuremberg Trials?\\n15. How did World War 2 lead to the creation of the United Nations?"}', 'name': 'return_questions'}}, response_metadata={'token_usage': {'completion_tokens': 207, 'prompt_tokens': 91, 'total_toke

## FALLBACK